In [6]:
import matplotlib.pyplot as plt
import cv2
import numpy as np
import glob
from PIL import Image, ImageDraw
import ffmpeg
%matplotlib inline

In [7]:
def slope(point1, point2):
    return(point2[1]-point1[1])/(point2[0]-point1[0])

def filtering_image(image1,topleft,topright,bottomleft,bottomright):
    # two line slopes 
    slope1=slope(bottomleft,topleft)
    slope2=slope(bottomright,topright)
    
    shp=image1.shape
    x,y=np.mgrid[0:shp[0],0:shp[1]]
    mask_img=(slope(bottomleft,[x,y]) < slope1) & (slope(bottomright,[x,y]) > slope2) &(x > topleft[0])
    flatlist=mask_img.astype(np.int8)
    #flatlist=map(lambda x: 255 if x else 0, mask_img.reshape(-1))
    tomask=np.array(flatlist).reshape(shp[0],shp[1])
    
    masked=cv2.bitwise_and(image1,image1, mask=tomask)
    return masked

def drawlines(m,raw, lines):
    #print(line1)
    todrawleft=cv2.line(raw,(int(lines[0][0][0]),int(lines[0][0][1])),(int(lines[0][1][0]),int(lines[0][1][1])), (255,255,0),5)
    todrawright=cv2.line(todrawleft,(int(lines[1][0][0]),int(lines[1][0][1])),(int(lines[1][1][0]),int(lines[1][1][1])), (255,255,0),5)
    return todrawright
    
    
def left_right_lines(lines):
    
    lline=[0,0,0,0]
    rline=[0,0,0,0]
    rcount,lcount=0,0
    for line in lines:
        p1=[float(line[0][0]),float(line[0][1])]
        p2=[float(line[0][2]),float(line[0][3])]
    
        if slope(p1,p2) > 0:
            lcount+=1
            lline=[lline[0]+p1[0],lline[1]+p1[1],lline[2]+p2[0],lline[3]+p2[1]]
        if slope(p1,p2) < 0:
            rcount+=1
            rline=[rline[0]+p1[0],rline[1]+p1[1],rline[2]+p2[0],rline[3]+p2[1]]
    rline=np.array(rline)/rcount
    lline=np.array(lline)/lcount
    return [[list(lline), list(rline)]]

def findfulllines(lrlines):
    # find the x intercept of both the lines
    
    p1=[float(lrlines[0][0][0]),float(lrlines[0][0][1])]
    p2=[float(lrlines[0][0][2]),float(lrlines[0][0][3])]
    s1=slope(p1,p2)
    c1=p2[1]-(p2[0]*s1)
    xintercept1=((540-c1)/s1)
    p2=[xintercept1,540.0]
    
    p3=[float(lrlines[0][1][0]),float(lrlines[0][1][1])]
    p4=[float(lrlines[0][1][2]),float(lrlines[0][1][3])]
    s2=slope(p3,p4)
    c2=p3[1]-(p3[0]*s2)
    xintercept2=((540-c2)/s2)
    p3=[xintercept2,540.0]
    return [[p1,p2],[p3,p4]]
    

In [8]:
# load the images


def Finding_lanes(images,videosignal):
    imagesfinal=[]
    for i in images:
        if videosignal:
            image=i
        else:
            image=cv2.imread(i)
        rgb=cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        #mask the white color
        low=np.array([200,200,200])
        high=np.array([255,255,255])
        white=cv2.inRange(rgb,low,high)

        # masking the yellow color
        lowy=np.array([190,190,0])
        highy=np.array([255,255,255])
        yellow=cv2.inRange(rgb,lowy,highy)
    
        #mask the yellow and white
        mask=np.bitwise_or(white, yellow)
        masked=cv2.bitwise_and(rgb,rgb,mask=mask)
        
        # Find the edges in images
        gray=cv2.cvtColor(masked, cv2.COLOR_RGB2GRAY)
        gray_smooth=cv2.GaussianBlur(gray, (13,13),0)
        edges=cv2.Canny(gray_smooth,50, 150)
    
        # removing all other features
        rows,cols=edges.shape[0],edges.shape[1]
        topleft=[rows*0.60, cols*0.45]
        topright=[rows*0.6, cols*0.6]
        bottomleft=[rows, cols*0.15]
        bottomright=[rows, cols*0.90]
        masked=filtering_image(edges,topleft,topright,bottomleft,bottomright)
        
        # extending the lines into full lines
        lines=cv2.HoughLinesP(image=masked,rho=1,theta=np.pi/180,threshold=20, minLineLength=20,maxLineGap=300)
        lrlines=left_right_lines(lines)
        fulllines=findfulllines(lrlines)
        imglines=drawlines(masked,i,fulllines)
        imagesfinal.append(imglines)
    return imagesfinal

def videotoimages(video_path):
    all_images=[]
    vidcap = cv2.VideoCapture(video_path)
    
    success,image = vidcap.read()
    count = 0
    while success:     
        success,image = vidcap.read()
        #print('Read a new frame: ', success)
        all_images.append(image)
        count += 1
    return all_images

def imagestovideo(images, output):
    frame = images[0]
    cv2.imshow('video',frame)
    height, width, channels = frame.shape
    
    fourcc = cv2.VideoWriter_fourcc(*'mp4v') # Be sure to use lower case
    out = cv2.VideoWriter(output, fourcc, 20.0, (width, height))
    
    for image in images:
        frame = image
        out.write(frame) # Write out frame to video
        cv2.imshow('video',frame)
        if (cv2.waitKey(1) & 0xFF) == ord('q'): # Hit `q` to exit
            break
    # Release everything if job is finished
    out.release()
    cv2.destroyAllWindows()

    print("The output video is {}".format(output))

In [9]:
video_input_path="/home/saichand/SDcar/Practice/P01_Find_lanes/data/test_videos/solidWhiteRight.mp4"
path="/home/saichand/SDcar/Practice/P01_Find_lanes/data/test_images/*.jpg"
output="/home/saichand/SDcar/Practice/P01_Find_lanes/data/output/1.mp4"

images=glob.glob(path)

imagesfromvideo=videotoimages(video_input_path)
imagesfromvideo=imagesfromvideo[:len(imagesfromvideo)-1]
final_images=Finding_lanes(imagesfromvideo, videosignal=True)

finalvideo=imagestovideo(final_images,output)





The output video is /home/saichand/SDcar/Practice/P01_Find_lanes/data/output/1.mp4
